## U-Net training for image segmentation

In [ ]:
import pyeddl.eddl as eddl
from pyeddl.tensor import Tensor

#### Load data

In [ ]:
ISBI_PATH = '/home/francisco/Documents/Universidad/5_Carrera/TFG_Computer_Science/datasets/isbi/'

In [ ]:
train_imgs = [ISBI_PATH + 'train/image_lq/'+str(i)+'.png' for i in range(30)]
train_masks= [ISBI_PATH + 'train/label_lq/'+str(i)+'.png' for i in range(30)]

In [ ]:
# Tensor.load(train_imgs[7])
# Tensor.load(train_masks[2])

In [ ]:
train_data  = [Tensor.load(img)            for img in train_imgs ] 
train_masks = [Tensor.load(msk).div(255.0) for msk in train_masks]

In [ ]:
# Tensor.getdata(train_data[0])
# Tensor.getdata(train_masks[0])

#### Create model

In [ ]:
from unet import unet

In [ ]:
# RuntimeError: Error allocating 288.00MB in ConvolDescriptor::build
#
# Solution 1:   https://stackoverflow.com/questions/57948003/how-to-increase-jupyter-notebook-memory-limit
# Solution 2:   Reduce image quality and shape to 256x256
net = unet(eddl.Input([1, 256, 256]))

In [ ]:
# Model build
eddl.build(
    net,
    eddl.nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, schedule_decay=0.004), 
    ["binary_cross_entropy"],
    ["dice"],
    eddl.CS_CPU()
)

In [ ]:
eddl.setlogfile(net, "unet.log")
eddl.summary(net)

In [ ]:
eddl.fit(net, train_data, train_masks, 2, 10)

#### Data preprocessing

Dealing with 512$\times$512 images requires too much memory for creating and training our model. In this first part, we are reducing image size to 256$\times$256

In [ ]:
from PIL import Image, ImageDraw

In [ ]:
ISBI_PATH = '/home/francisco/Documents/Universidad/5_Carrera/TFG_Computer_Science/datasets/isbi/'

In [ ]:
train_imgs = [ISBI_PATH + 'train/image/'+str(i)+'.png' for i in range(30)]
train_masks= [ISBI_PATH + 'train/label/'+str(i)+'.png' for i in range(30)]

In [ ]:
for i in range(30):
    image_input  = ISBI_PATH + 'train/image/'     + str(i) + '.png'
    mask_input   = ISBI_PATH + 'train/label/'     + str(i) + '.png'
    image_output = ISBI_PATH + 'train/image_lq/' + str(i) + '.png'
    mask_output  = ISBI_PATH + 'train/label_lq/' + str(i) + '.png'
    
    img = Image.open(image_input)
    msk = Image.open(mask_input)
    
    img = img.resize((img.size[0]//2, img.size[1]//2))
    msk = msk.resize((msk.size[0]//2, msk.size[1]//2))
    
    img = img.save(image_output)
    msk = msk.save(mask_output)